## Slicing

Slicing is a way to select some elements of an object and it works with arrays, strings and tuples. Slicing operations create a copy by default.
<pre> x[1:10] </pre>
OR
<pre>view(x, 1:10)</pre>

**view()** Like getindex, but returns a lightweight array that lazily references (or is effectively a view into) the parent array A at the given index or indices inds instead of eagerly extracting elements or constructing a copied subset. 


In [26]:
arr1 = [2,3,5,7,11,13,17]

str1 = "abcdefg"

tup1 = 1,2,3,4,5,6,7

(1, 2, 3, 4, 5, 6, 7)

#### Basic Range Examples 

In [2]:
arr1[1:3]

3-element Vector{Int64}:
 2
 3
 5

In [ ]:
str1[4:length(str1)]

"defg"

In [ ]:
tup1[1:5]

In [28]:
tup1 |> first
# same as calling
first(tup1)

1


If you are doing many operations on the slice, this can be good for performance because it is more efficient to work with a smaller contiguous copy than it would be to index into the original array.

<b>@view</b> Transform the indexing expression into equivalent view cal

<b>@views</b> converts string slices into SubString views.

A view is a datastructure that acts like an array, but underlying data is actually part of another array
<pre> @views x[1:10]</pre>

Sometimes make a copy of the data is faster and sometimes using a view is faster

if you are just doing a few simple operations on the slice, the cost of the allocation and copy operations can be substantial

In [1]:
x = rand(10^6)

@views fview(x) = sum(x[2:end-1])
fcopy(x) = sum(x[2:end-1])

@time fview(x)
@time fcopy(x)

  0.122296 seconds (206.75 k allocations: 10.964 MiB, 98.77% compilation time)
  0.117463 seconds (104.64 k allocations: 13.111 MiB, 13.31% gc time, 80.62% compilation time)


499527.87907923135

#### Copying data is not always bad

Arrays are stored contiguously in memory, lending themselves to CPU vectorization and fewer memory accesses due to caching. 

These are the same reasons that it is recommended to access arrays in column-major order

<h5>Copying irregularly-accessed data into a contiguous array before repeated access it can result in a large speedup.</h5>

Here, a matrix is being accessed at randomly-shuffled indices before being multiplied. Copying into plain arrays speeds up the multiplication even with the added cost of copying and allocation.

In [ ]:
using Random
A = randn(3000, 3000)
x = randn(2000)

inds = shuffle(1:3000)[1:2000]

function iterated_neural_network(A, x, depth)
    for _ in 1:depth
        x /= max.(0, A * x)
    end
    argmax(x)
end

@time iterated_neural_network(view(A, inds, inds), x, 10.)
@time iterated_neural_network(A[inds, inds], x, 10)
## THROWING ERROR ##
## DO NOT RUN CELL ##

##### Provided there is enough memory, the cost of copying the view to an array is outweighed by the speed boost from doing the repeated matrix multiplications on a contiguous array.

if your application involves many small (< 100 element) arrays of fixed sizes (i.e. the size is known prior to execution), then you might want to consider using the StaticArrays.jl package. 

## Built In Function: any() and all()

The functions any() and all() take an **iterable** argument.

 **any()** Test whether any elements of a boolean collection are true, returning true as soon as the first true value in itr is encountered (short-circuiting)

**all()** Test whether all elements of a boolean collection are true, returning false as soon as the first false value in itr is encountered (short-circuiting)


In [23]:
a = [true,true,true,true]
print(all(a))
a = [true false; false true]
all!([1; 1], a)

true

2-element Vector{Int64}:
 0
 0

In [24]:
print(any(i->(4<=i<=6), [3,5,7]))
A = [true false; true false]
any!([1; 1], A)

true

2-element Vector{Int64}:
 1
 1

In [ ]:
# List for exercises

ex1 = [2,3,5,7,11,13,17,19,23]

# String for exercises 3

ex2 = "abcdefghijk"

### Exercises

<ol>
    <li>Find a slice of ex1 that returns [23, 13, 5].</li>
    <li>Find a slice of ex1 that returns [7, 17].</li>
    <li>Find a slice of ex2 that returns 'def'.</li>
</ol>

## Dictionaries
the standard dictionary implementation uses <b>hash</b> as the hashing function for the key, and <b>isequal</b> to determine equality. 
<pre>AbstractDict{K, V}</pre>
Supertype for dictionary-like types with keys of type K and values of type V. Dict, IdDict and other types are subtypes of this. 

<pre> Dict{K,V}()</pre>
 constructs a hash table with keys of type K and values of type V. Keys are compared with isequal and hashed with hash.


In [1]:
d1 = Dict([("A", 1), ("B", 2)])

Dict{String, Int64} with 2 entries:
  "B" => 2
  "A" => 1

In [2]:
Dict("A"=>1, "B"=>2)

Dict{String, Int64} with 2 entries:
  "B" => 2
  "A" => 1

In [12]:
get(d1, "1", 3) # defaults to 3 if no mapping for the key is present

get!(d1, "3", 3) #  if no mapping for the key is present ! indicates it will store in place
d1

Dict{String, Int64} with 3 entries:
  "B" => 2
  "A" => 1
  "3" => 3

In [13]:
d1["newkey"] = 0
d1

Dict{String, Int64} with 4 entries:
  "newkey" => 0
  "B"      => 2
  "A"      => 1
  "3"      => 3

In [17]:
getkey(d1, "newkey", "defaultkey")

"newkey"

In [18]:
getkey(d1, "newke", "defaultkey")

"defaultkey"

#### More Dictonary functions

In [19]:
delete!(d1, "B")

Dict{String, Int64} with 3 entries:
  "newkey" => 0
  "A"      => 1
  "3"      => 3

In [20]:
pop!(d1)
d1

Dict{String, Int64} with 2 entries:
  "A" => 1
  "3" => 3

In [6]:
keys(d1)

KeySet for a Dict{String, Int64} with 2 entries. Keys:
  "B"
  "A"

In [7]:
values(d1)

ValueIterator for a Dict{String, Int64} with 2 entries. Values:
  2
  1

In [8]:
pairs(d1)

Dict{String, Int64} with 2 entries:
  "B" => 2
  "A" => 1

In [23]:
a = Dict("foo" => 0.0, "bar" => 42.0)
b = Dict("baz" => 17, "bar" => 4711)

merge(b,a)

merge((a=1, b=2, c=3), (b=4, d=5))

mergewith(+, a, b)

Dict{String, Float64} with 3 entries:
  "bar" => 4753.0
  "baz" => 17.0
  "foo" => 0.0

In [9]:
IdDict(true => "yes", 1 => "no", 1.0 => "maybe")

IdDict{Any, String} with 3 entries:
  1.0  => "maybe"
  1    => "no"
  true => "yes"

## Inclusion Operators: in and not in

The **in** or  **∈** and **∉** operators can be used to check for inclusion in sets, arrays, tuples and dictionaries.
<pre>
in(item, collection) -> Bool
∈(item, collection) -> Bool
∉(item, collection) -> Bool
∌(collection, item) -> Bool
</pre>

In [38]:
a = 1:3:20

println(4 in a)

println(5 in a)

[1, 2] ∈ [2, 3]

true
false


false

In [29]:
1 ∉ 2:4

true

In [49]:
("B"=>1) ∉ Dict("A"=>1, "B"=>2)

true

## Exercises

<ol>
    <li>Two dictionaries, dct5 and dct6, are defined in a cell below. Use a for loop to add the items in dct5 to dct6.  <b>Note</b>: there is a dictionary method update() that can do this, but I want a solution that uses a for loop.</li>
    <li>Use a for loop to create a dictionary dct7 whose keys are the integers 1,2,...,10 and whose values are the cubes of the keys.</li>

In [55]:
dct5 = Dict(2 => 4, 3 => 9, 4 => 16)

dct6 = Dict(5 => 25, 6 => 36, 7 => 49)

Dict{Int64, Int64} with 3 entries:
  5 => 25
  6 => 36
  7 => 49

## Comprehensions

Comprehensions provide a general and powerful way to construct arrays. Comprehension syntax is similar to set construction notation in mathematics
<pre>A = [ F(x, y, ...) for x=rx, y=ry, ... ]</pre>
The meaning of this form is that F(x,y,...) is evaluated with the variables x, y, etc. taking on each value in their given list of values. Values can be specified as any iterable object, but will commonly be ranges like 1:n or 2:(n-1), or explicit arrays of values like [1.2, 3.4, 5.7]

**Example: create an array containing the squares of the integers 1 through 10.**

In [58]:
sq1 = []

for i in range(1,10)
    append!(sq1, i^2)
end
sq1

10-element Vector{Any}:
   1
   4
   9
  16
  25
  36
  49
  64
  81
 100

In [59]:
sq2 = [i^2 for i in range(1,10)]

10-element Vector{Int64}:
   1
   4
   9
  16
  25
  36
  49
  64
  81
 100

In [61]:
# we can also use map do syntax for the same thing
# map will preserve the type of the collection
# where as the comprehension will always return an Array
sq3 = map(1:10) do i
    i^2
end
# WHEN IN DOUBT, start with a comprehension and switch to 
# map do if the expression is too complex

10-element Vector{Int64}:
   1
   4
   9
  16
  25
  36
  49
  64
  81
 100

## Exercises

<ol>
    <li>Use a list comprehension to produce the list [-3,1,5,9,13]</li>
    <li>Use a list comprehension to produce the list [1,-2,3,-4,5,-6].</li>
    <li>Use a for loop to produce the list [1,2,4,7,11,16]
</ol>